**ДЗ 2**

**Матяш Дарья БКЛ-181**

Для начала включим проверку PEP8:

In [95]:
%load_ext pycodestyle_magic
%pycodestyle_on

Импортируем все нужные модули

In [96]:
import time
from pymystem3 import Mystem
import json
import nltk
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
from collections import Counter

"Парсим" текст с помощью Mystem и замеряем время работы
 **Важно!** я клянусь, у меня ВСЁ РАБОТАЛО, А ПОТОМ РЕЗКО текст тхт перестал восприниматься как что-то нормальное

In [97]:
m = Mystem()
with open('groza.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    start = time.time()
    ana = m.analyze(text)
    end = time.time()
    print('Время работы составило: ', end - start, 'сек.')

Сохраняем результат в json

In [100]:
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(ana, f)

Токенизируем текст с помощью nltk

In [101]:
# токенизируем текст, приводим к нижнему регистру
# и оставляем только последовательности из букв,
# т.е. все токены, где были знаки препинания и числа, исчезнут
words = [w.lower() for w in word_tokenize(text) if w.isalpha()]

Разбираем слова с помощью pymorphy и замеряем время работы

In [102]:
morph = MorphAnalyzer()
ans_time = 0
for i in range(len(words)):
    start = time.time()
    morph.parse(words[i])
    end = time.time()
    dif = end - start
    ans_time += dif
print('Время работы составило: ', ans_time, 'сек.')

In [103]:
ans_d = {}
for i in range(len(words)):
    w = morph.parse(words[i])[0]
    ans_d[w.normal_form] = w.tag.POS

Сохраняем результат в json

In [104]:
with open('data_2.json', 'w', encoding='utf-8') as f_2:
    json.dump(ans_d, f_2)

**Какую долю слов составляет каждая часть речи?**

In [113]:
number_of_words = len(words)
ans_pos = {}
verbs = []
adverbs = []
for i in range(len(words)):
    w = morph.parse(words[i])[0]
    if w.tag.POS in ans_pos:
        ans_pos[w.tag.POS] += 1
    else:
        ans_pos[w.tag.POS] = 1
    if w.tag.POS == 'VERB':
        verbs.append(w.normal_form)
    elif w.tag.POS == 'ADVB':
        adverbs.append(w.normal_form)
        verbs.append(w.normal_form)
print('Доля части речи в тексте', '\n')
print('Часть речи:', '\t', 'Её доля в тексте:', '\n')
for key in ans_pos.keys():
    print(key, '\t', ans_pos[key] / number_of_words)

**Топ-20 (по частотности) глаголов и наречий**

In [114]:
top_10_verbs = Counter(verbs).most_common(20)
print('Топ-20 частотных глаголов:', '\n')
count = 1
for tup in top_10_verbs:  # выводим топ-10 глаголов
    print(str(count) + ')', '\t', tup[0])
    count += 1

In [107]:
top_10_adverbs = Counter(adverbs).most_common(20)
print('Топ-20 частотных наречий:', '\n')
count = 1
for tup in top_10_adverbs:
    print(str(count) + ')', '\t', tup[0])
    count += 1

**Топ-25 биграмм и триграмм**

**Примечание:** я считала би- и три-граммы именно по леммам, а не про формам, представленным в тексте

In [108]:
words_lemmas = []
for w in ana:   # ходим по размеченному тексту
    if 'analysis' in w.keys():
        if w['analysis'] != []:
            lemma = w['analysis'][0]['lex']
            words_lemmas.append(lemma)   # сохраняем в список все леммы
bigram = list(nltk.bigrams(words_lemmas))   # создаем список в биграммами
top_25_bigrams = Counter(bigram).most_common(25)
count = 1
print('Топ-25 биграм:', '\n')
for tup in top_25_bigrams:
    print(tup)
    print(str(count) + ')', '\t', list(tup[0])[0], '\t', list(tup[0])[1])
    count += 1

In [109]:
bigram = list(nltk.bigrams(words_lemmas))   # создаем список в биграммами
top_25_bigrams = Counter(bigram).most_common(25)
count = 1
for tup in top_25_bigrams:
    print(tup)
    print(str(count) + ')', '\t', list(tup[0])[0], '\t', list(tup[0])[1])
    count += 1

In [110]:
trigram = list(nltk.trigrams(words_lemmas))  # создаем список с триграммами
top_25_trigrams = Counter(trigram).most_common(25)
count = 1
print('Топ-25 триграм:', '\n')
for tup in top_25_trigrams:  # выводим топ-25 биграм
    list_tup = list(tup[0])
    print(str(count) + ')', *list_tup[:2])
    count += 1